# Reserve and configure FABRIC resources for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [2]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [3]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [4]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test-1'
#slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [17]:
exp_requires = {'core': 2*n_endpoints*4+32, 'nic': 2*n_endpoints*1, 'connectx5': 1}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) and
        (fablib.resources.get_component_available(site_name, 'SmartNIC-ConnectX-5') > 1.2*exp_requires['connectx5'])   ):
        break

fablib.show_site(site_name)

Name,DALL
State,Active
Address,"1950 N Stemmons Fwy,Dallas TX 75207"
Location,"(32.79908160563668, -96.82069027727609)"
Hosts,3
CPUs,6
Cores Available,290
Cores Capacity,384
Cores Allocated,94
RAM Available,1306
RAM Capacity,1506


'<pandas.io.formats.style.Styler object at 0x7f9996347b80>'

In [18]:
# this cell sets up the hosts
slice.add_node(name='router', site=site_name, cores=32, ram=256, disk=25, image='default_ubuntu_22')

sender_names = ["sender-"+str(i) for i in range(n_endpoints)]
for n in sender_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=25, image='default_ubuntu_18')
    
receive_names = ["receiver-"+str(i) for i in range(n_endpoints)]
for n in receive_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=25, image='default_ubuntu_18')

In [19]:
# this cell sets up the network links
nets = [
    {"name": "link-sender",    "nodes": sender_names,  "idx": 0},
    {"name": "link-receiver",  "nodes": receive_names, "idx": 1}
]

router_iface = slice.get_node('router').add_component(model="NIC_ConnectX_5", name='link').get_interfaces()

for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ] + [router_iface[n["idx"]]]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [20]:
slice.submit()


Retry: 20, Time: 5004 sec


ID,6011009b-bb9e-41ac-bdd0-44c99bc8b9ea
Name,bottleneck-10-test-1
Lease Expiration (UTC),2023-08-20 05:29:28 +0000
Lease Start (UTC),2023-08-19 05:29:29 +0000
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9419ffea-1a24-4b35-bf7c-7448b540012e,receiver-0,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feb2:1562,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feb2:1562,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
3623c197-2a97-4c12-b9ec-436db625372c,receiver-1,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fed6:d5f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fed6:d5f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a11f0b62-80e5-4de3-93e0-243d90b7df68,receiver-2,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe83:105e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe83:105e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
63691569-7d15-4de6-8552-451e5a120af1,receiver-3,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feb0:a8d1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feb0:a8d1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
47ffbc0a-1d90-4bc6-a205-1a0fc4a1d8ba,receiver-4,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe81:e380,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe81:e380,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
36338ab2-4966-420e-a224-f33b86172089,receiver-5,4,32,100,default_ubuntu_18,qcow2,dall-w1.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:feaf:74ea,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:feaf:74ea,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d2d7d672-4e77-437b-939d-e08892f9298b,receiver-6,4,32,100,default_ubuntu_18,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe4c:1d4b,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe4c:1d4b,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4d666118-b16f-4fdc-81ce-17e29e3ba8fc,receiver-7,4,32,100,default_ubuntu_18,qcow2,dall-w1.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fed2:440,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fed2:440,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d6a025b6-e81b-4d15-af5a-3825df0f92ae,receiver-8,4,32,100,default_ubuntu_18,qcow2,dall-w1.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe1c:3021,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe1c:3021,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
fb4c7d53-1cd8-4d23-ac0c-1da07e26bc43,receiver-9,4,32,100,default_ubuntu_18,qcow2,dall-w1.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe36:d4d4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe36

ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
af64403c-6b68-4e36-819c-c750de56a1ed,link-receiver,L2,L2Bridge,DALL,None,None,Active,
2bebbdea-7086-4eb5-97b6-42d0eb30acd1,link-sender,L2,L2Bridge,DALL,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
router-link-p1,p1,router,link-sender,25,config,,B8:CE:F6:AF:85:56,ens7np0,ens7np0,None,6
router-link-p2,p2,router,link-receiver,25,config,,B8:CE:F6:AF:85:57,ens8np0,ens8np0,None,6
sender-0-link-sender-p1,p1,sender-0,link-sender,100,config,,0E:9A:4D:32:E8:CF,ens7,ens7,None,4
sender-1-link-sender-p1,p1,sender-1,link-sender,100,config,,0E:E0:82:3C:73:8B,ens7,ens7,None,4
sender-2-link-sender-p1,p1,sender-2,link-sender,100,config,,0E:95:35:10:C0:90,ens7,ens7,None,4
sender-3-link-sender-p1,p1,sender-3,link-sender,100,config,,0E:C4:E1:FB:0F:7D,ens7,ens7,None,4
sender-4-link-sender-p1,p1,sender-4,link-sender,100,config,,1A:23:5D:F7:79:75,ens7,ens7,None,4
sender-5-link-sender-p1,p1,sender-5,link-sender,100,config,,0E:D9:FB:36:22:79,None,None,None,4
sender-6-link-sender-p1,p1,sender-6,link-sender,100,config,,12:77:DB:DF:31:12,ens7,ens7,None,4
sender-7-link-sender-p1,p1,sender-7,link-sender,100,config,,12:81:7E:16:C3:FC,ens7,ens7,None,4



Time to print interfaces 5129 seconds


'6011009b-bb9e-41ac-bdd0-44c99bc8b9ea'

## Configure resources

In [8]:
slice.get_state()

'StableOK'

In [7]:
slice = fablib.get_slice(name=slice_name)

In [9]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice ........................................................ Timeout exceeded (1800 sec). Slice: bottleneck-10-test-1 (StableOK)


Exception:  Timeout exceeded (1800 sec). Slice: bottleneck-10-test-1 (StableOK)

Bring up all of the network interfaces:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

Assign addresses to router interfaces and enable forwarding:

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_sender = slice.get_interface('router-link-p1')
if_sender.ip_addr_add(addr="10.10.1.1", subnet=IPv4Network("10.10.1.0/24"))
if_receive = slice.get_interface('router-link-p2')
if_receive.ip_addr_add(addr="10.10.2.1", subnet=IPv4Network("10.10.2.0/24"))

slice.get_node(name='router').execute("sudo sysctl -w net.ipv4.ip_forward=1")

Assign addresses to host (sender and receiver) interfaces and set up routes:

In [ ]:
for i in range(n_endpoints):
    if_name = slice.get_interface('sender-' + str(i) + '-link-sender-p1')
    if_name.ip_addr_add(addr="10.10.1.1" + str(i) , subnet=IPv4Network("10.10.1.0/24"))
    slice.get_node(name='sender-' + str(i)).ip_route_add(subnet=IPv4Network("10.10.2.0/24"), gateway="10.10.1.1")
    
for i in range(n_endpoints):
    if_name = slice.get_interface('receiver-' + str(i) + '-link-receiver-p1')
    if_name.ip_addr_add(addr="10.10.2.1" + str(i) , subnet=IPv4Network("10.10.2.0/24"))
    slice.get_node(name='receiver-' + str(i)).ip_route_add(subnet=IPv4Network("10.10.1.0/24"), gateway="10.10.2.1")

Also install `iperf3` on sender and receiver hosts:

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
from ipaddress import ip_address, IPv6Address
for n in sender_nodes:
    if type(ip_address(n.get_management_ip())) is IPv6Address:
        n.upload_file('nat64.sh', 'nat64.sh')
        stdout, stderr = n.execute(f'chmod +x nat64.sh && ./nat64.sh')
    n.execute("sudo apt-get update; sudo apt-get -y install build-essential ")
    n.execute_thread("sudo modprobe tcp_bbr")
    n.execute_thread("sudo git clone https://github.com/vinitaparasrampuria/iperf.git /iperf; sudo chmod +x /iperf/configure; sudo bash /iperf/configure;\
                     sudo make; sudo make check; sudo make install; sudo ldconfig")
              
for n in receiver_nodes:
    n.execute_thread("sudo apt update; sudo apt -y install iperf3;  sudo modprobe tcp_bbr")

In [ ]:
for n in sender_nodes:
    n.execute("sudo apt-get update; sudo apt-get install -y python3-pip ethtool netcat moreutils ",quiet=True)
    n.execute("sudo python3 -m pip install scikit-learn numpy pandas matplotlib seaborn", quiet=True)


In [ ]:
for n in sender_nodes:
    n.execute("iperf3 -version")

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 4 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)